In [12]:
import sys

sys.path.append("/home/nikola/projects/deepRec/recsys")
print(sys.path)

['/home/nikola/projects/deepRec/recsys/vae', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/nikola/projects/deepRec/recsys/env/lib/python3.10/site-packages', '/home/nikola/projects/deepRec/recsys', '/tmp/tmplcdvaace', '/home/nikola/projects/deepRec/recsys']


In [14]:
from core.dataset import DatasetFactory
from dataset import Dataset
from torch.utils.data import DataLoader

cutoff = lambda x: 1.0 if x >= 3.5 else 0.0  # function for generating implicit ratings

dataset = DatasetFactory("../data/ml-100k/u.data", sep="|").create_dataset(
    filters=lambda df: df.groupby("user").filter(lambda x: len(x) >= 20)
)
print(dataset.n_items, dataset.n_users)
train, valid, test = dataset.tvt(mode="user")

train = Dataset(train, dataset.n_features, cutoff)
valid = Dataset(valid, dataset.n_features, cutoff)
test = Dataset(test, dataset.n_features, cutoff)

train = DataLoader(train, batch_size=100, num_workers=12)
valid = DataLoader(valid, batch_size=1, num_workers=12)
test = DataLoader(test, batch_size=1, num_workers=12)

1682 943


In [ ]:
from model import Model

model = Model(dataset.n_items, 200)

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

trainer = pl.Trainer(
    max_epochs=200,
    log_every_n_steps=20,
    callbacks=[EarlyStopping(monitor="ndcg@100", mode="max", patience=10)],
)

In [ ]:
trainer.fit(model, train, valid)

In [ ]:
trainer.test(model, test)